# Final Project: Black Friday Excel Data

In [8]:
import openpyxl as xl
from tax_calculator import tax_calculator

# pprint prints dictionaries a bit more nicely than print
from pprint import pprint

## Part 1

First, read in maven_ski_shop_data.xlsx

Then, assign the orders worksheet to an intuitive variable name. 

In [10]:
wb = xl.load_workbook("maven_ski_shop_data.xlsx")

orders = wb['Orders_Info']

### Column Printer Function

Since we'll be working with Excel frequently, let's create a helper function to print all the rows a specified column.

This will help us view imported data without looking at Excel, and review the data we've written into columns without needing to save a file to review the changes.

Your function should take two arguments:

* Worksheet where data is located
* Column Letter of data to print

Your function should:

* Print Cell Coordinate (e.g. A1, D4).
* Print the contents of the cell.

Then call your function twice:

* Once to print Order ID (column A in 'Orders_Info')
* Once to print subtotal (column D in 'Orders_Info')

In [12]:
def column_printer(worksheet, column):
    for i, cell in enumerate(orders[f'{column}'], start = 1):
        print(f'{column}{i} {cell.value}')
        

In [13]:
column_printer(orders, 'A')

A1 Order_ID
A2 100000
A3 100001
A4 100002
A5 100003
A6 100004
A7 100005
A8 100006
A9 100007
A10 100008
A11 100009
A12 100010
A13 100011
A14 100012
A15 100013
A16 100014
A17 100020
A18 100021
A19 100022
A20 100023
A21 100024
A22 100025
A23 100026
A24 100027
A25 100031
A26 100032
A27 100033
A28 100034


In [14]:
column_printer(orders, 'D')

D1 Subtotal
D2 15.98
D3 899.97
D4 799.97
D5 117.96
D6 5.99
D7 599.99
D8 24.99
D9 1799.94
D10 99.99
D11 254.95
D12 29.98
D13 99.99
D14 25.98
D15 649.98
D16 89.99
D17 119.99
D18 599.99
D19 649.98
D20 24.99
D21 99.99
D22 99.99
D23 5.99
D24 24.99
D25 999.96
D26 99.99
D27 399.97
D28 89.99


### Order Data Dictionary

Create a dictionary with all of the information contained in the 'Orders_Info' Worksheet.

* The dictionary keys should be Order IDs (Column A)
* The values should a list storing the data in the rest of the columns.
* The columns in the list stored as values should be B, C, D, G, and H (after converting to list). 
* For example: The first order, column H should be the nested list: [10001, 10002]


A few notes:

* Convert the Items_Ordered field into a list (we covered a helpful string method for this)
* This can be done with a dictionary comprehension
* DO NOT INCLUDE EXCEL COLUMN HEADERS

In [16]:
order_dict = {
    orders[f'A{order}'].value: [
        orders[f'B{order}'].value,
        orders[f'C{order}'].value,
        orders[f'D{order}'].value,
        orders[f'G{order}'].value,
        str(orders[f'H{order}'].value).split(', ')
    ] 
    for order in range(2, orders.max_row + 1)
}

In [17]:
pprint(order_dict)

{100000: ['C00004', '11/26/2021', 15.98, 'Sun Valley', ['10001', '10002']],
 100001: ['C00007', '11/26/2021', 899.97, 'Stowe', ['10008', '10009', '10010']],
 100002: ['C00015',
          '11/26/2021',
          799.97,
          'Mammoth',
          ['10011', '10012', '10013']],
 100003: ['C00016',
          '11/26/2021',
          117.96,
          'Stowe',
          ['10002', '10003', '10004', '10006']],
 100004: ['C00020', '11/26/2021', 5.99, 'Sun Valley', ['10001']],
 100005: ['C00010', '11/26/2021', 599.99, 'Mammoth', ['10010']],
 100006: ['C00006', '11/26/2021', 24.99, 'Mammoth', ['10004']],
 100007: ['C00001',
          '11/26/2021',
          1799.94,
          'Mammoth',
          ['10008', '10008', '10009', '10009', '10009', '10010', '10010']],
 100008: ['C00003', '11/26/2021', 99.99, 'Sun Valley', ['10005']],
 100009: ['C00014',
          '11/26/2021',
          254.95,
          'Sun Valley',
          ['10002', '10003', '10004', '10006', '10007']],
 100010: ['C00001', '11/

### Sales Tax Calculation (Last time!)

We need to calculate the sales tax and total amount owed for every order in this sheet.

* If location is Sun Valley, apply a sales tax of 8%
* If location is Mammoth, apply a sales tax of 7.75%.
* If location is Stowe, apply a sales tax of 6%

Use the tax_calculator function to apply sales tax to each subtotal.

**Insert** the calculated sales tax and total amounts into your customer dictionary immediately after subtotal.

**Tip**: Figure out how to do this once and then loop through the column.

In [19]:
for order in order_dict.values():
    if order[3] == "Sun Valley":
        transaction = tax_calculator(order[2], .08)
    elif order[3] == "Mammoth":
        transaction = tax_calculator(order[2], .0775)
    else:
        transaction = tax_calculator(order[2], .06)
    order.insert(3, transaction[1])
    order.insert(4, transaction[2])

###### pprint(order_dict)

### Write Sales Tax and Total Into the Excel Sheet


Great job! Now we just need to write this data into the workbook.

Write the sales tax and total you just calculated into the workbook, then save!

Call this workbook 'maven_ski_shop_data_fixed'.

**Tip:** There are a few ways to do this. As always, be patient, solve one step at a time.

In [34]:
i = 2
for order in order_dict.values():
    orders[f'E{i}'].value = order[3]
    orders[f'F{i}'].value = order[4]
    i += 1

In [36]:
wb.save('maven_ski_shop_data_fixed.xlsx')

## Part 2

### Analysis Time!


Now that we've fixed the data - it's time to perform analysis on our sales.

Our starting point will be the order_dict we created, after we added the taxes and totals columns.

Our first step will be to write a function that calculates the sum of a 'column' of data in our dictionary.

A 'column' for example, would be subtotals, which is at index 2 in the list stored as our dictionary values.

Your function should take the following arguments:

* Column Index (the index of the data of interest in the list stored in our dictionary values)
* Dictionary Name

It should output:

* The sum of values in the column (rounded to two decimal places)

Assume only numeric values will be in the column (You can develop cleaning logic later :D)

**Tip:** Use a list comprehension to retrieve the values of interest.

In [62]:
def column_sum(index, dictionary):
    sum = 0
    for order in dictionary.values():
        sum += order[index]
    return round(sum, 2)

#### Sum The Subtotal, Tax, and Total Columns 

Now that we have our column sum function, calculate the sum of:

* Subtotals
* Taxes
* Totals

In [64]:
print(column_sum(2, order_dict))
print(column_sum(3, order_dict))
print(column_sum(4, order_dict))

8731.47
617.2
9348.67


### What is the average of our subtotals?

Calculate the average value of our transactions.

Remember that each entry in your dictionary is one order.

In [66]:
avg = round(column_sum(2, order_dict) / len(order_dict), 2)
avg

323.39

### How many unique customers did we have?

Calculate the total number of unique customers in our sales data.

Then calculate the number of orders per customer (total orders / unique customers)

In [76]:
len(set([order[0] for order in order_dict.values()]))

19

### How many items in total did we sell?

Calculate the total number of items we sold in across all orders. 

This information is in Column H, which should be the last element in order_dict's values.

In [90]:
sum([len(order[6]) for order in order_dict.values()])

54

### Sales By Location

Calculate the sum of subtotals by location.

Create a dictionary to store them, where location is the key, and revenue for that location is the value.

A few steps to consider:

* Loop through your dictionary
* build a dictionary as you go with location as key
* increment revenue every time a transaction matches the location.


Your output should look like 
{'Location1': sum of subtotals for 'Location1'}

With an entry for each location.

In [92]:
location_sums = {}
for data in order_dict.values():
    location = data[5]
    if location not in location_sums:
        location_sums[location] = 0
    location_sums[location] += data[2]

location_sums

{'Sun Valley': 1268.84,
 'Stowe': 3582.8199999999993,
 'Mammoth': 3879.8099999999995}

### Challenge: Aggregator Function

Now that you've summed revenue by category (you just summed by location), can you write a function to generalize calculating a sum of a column, grouped by the unique values in another column? (for example, sum of totals by date or customer_id).

Your function should take the following arguments:
* index of the 'column' (index position in order_dict) to group by
* index of the 'column' (index position in order_dict) to sum by category
* the dictionary where the data is located (assume the same structure as order_dict.

It should return:
* A dictionary with the categories as keys, and the sum by category as value.

Once you've done so use your function to sum totals by date and customer_id.

In [100]:
def aggregator(idx1, idx2, dictionary):
    return_dict = {}
    for data in order_dict.values():
        group_by = data[idx1]
        if group_by not in return_dict:
            return_dict[group_by] = 0
        return_dict[group_by] += data[idx2]
    return return_dict

In [102]:
# index 1 = date, index 4 = total
aggregator(1, 4, order_dict)

{'11/26/2021': 5915.179999999999,
 '11/27/2021': 1714.25,
 '11/28/2021': 1719.2400000000002}

In [104]:
# index 0 = customer_id
aggregator(0, 4, order_dict)

{'C00004': 146.85,
 'C00007': 953.97,
 'C00015': 861.97,
 'C00016': 220.43,
 'C00020': 6.47,
 'C00010': 1077.46,
 'C00006': 53.86,
 'C00001': 2079.48,
 'C00003': 107.99,
 'C00014': 275.35,
 'C00005': 28.06,
 'C00008': 794.97,
 'C00013': 97.19,
 'C00017': 635.99,
 'C00019': 701.98,
 'C00002': 26.49,
 'C00021': 107.74,
 'C00022': 6.47,
 'C00018': 1165.95}